In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
pngimg = Image.open("./dataset/Annotations/IMG20230927100144.png")
rgbimg = Image.open("./dataset/images/IMG20230927100144.jpg")

In [4]:
pngimg_array = np.array(pngimg)
unique,count=np.unique(pngimg_array,return_counts=True)
dict(zip(unique,count))

{0: 12569874, 2: 13038}

In [8]:
pngimg_array[pngimg_array == 2] = 1

In [5]:
# 定义矩形区域的左上角和右下角坐标
start_row, start_col = 100, 100
end_row, end_col = 300, 300

# 将矩形区域的值设置为1
pngimg_array[start_row:end_row+1, start_col:end_col+1] = 1

unique,count=np.unique(pngimg_array,return_counts=True)
dict(zip(unique,count))

{0: 12529473, 1: 40401, 2: 13038}

In [6]:
import time

def normalize_arrays(tuple_of_arrays,x_max,y_max):
    sample_size = 50
    array_x, array_y = tuple_of_arrays
    array1_normalized = array_x / x_max
    array2_normalized = array_y / y_max
    point_set = np.column_stack([np.around(array2_normalized, decimals=6),np.around(array1_normalized, decimals=6)])
    if point_set.shape[0] > sample_size:
        return point_set[np.random.choice(point_set.shape[0], size=sample_size, replace=False)]
    else:
        return point_set

def label_to_txt(array, file_path, a):
    with open(file_path , 'w') as file:
        file.write(str(a))
        for row in array:
            file.write(' '+' '.join(map(str, row)) )
            
def add_to_txt(array,file_path ,a):
    with open(file_path , 'a') as file:
        file.write('\n')
        file.write(str(a))
        for row in array:
            file.write(' '+' '.join(map(str, row)) )
#多个物体写入txt

def png_to_label(pngimg_array, file_path):
    leash = np.where(pngimg_array == 1)
    dog = np.where(pngimg_array == 2)
    x_max,y_max = pngimg_array.shape
    if len(leash[0])==0:
        edges = cv2.Canny(np.uint8(pngimg_array * 255), 0, 1)
        data_point_dog = np.where(edges == 255)
        data_point_dog = normalize_arrays(data_point_dog,x_max,y_max)
        label_to_txt(data_point_dog,file_path,2)
    elif len(dog[0])==0:
        edges = cv2.Canny(np.uint8(pngimg_array * 255), 0, 1)
        data_point_leash = np.where(edges == 255)
        data_point_leash = normalize_arrays(data_point_leash,x_max,y_max)
        label_to_txt(data_point_leash,file_path,1)
    else:
        zero_pngimg_array1 = np.zeros_like(pngimg_array, dtype=np.uint8)
        zero_pngimg_array2 = np.zeros_like(pngimg_array, dtype=np.uint8)
        zero_pngimg_array1[pngimg_array == 1] = 1
        zero_pngimg_array2[pngimg_array == 2] = 2
        edges1 = cv2.Canny(np.uint8(zero_pngimg_array1 * 255), 0, 1)
        edges2 = cv2.Canny(np.uint8(zero_pngimg_array2 * 255), 0, 1)
        data_point_leash = np.where(edges1 == 255)
        data_point_dog = np.where(edges2 == 255)
        data_point_dog = normalize_arrays(data_point_dog,x_max,y_max)
        data_point_leash = normalize_arrays(data_point_leash,x_max,y_max)
        label_to_txt(data_point_leash,file_path,1)
        add_to_txt(data_point_dog,file_path,2)



t0 = time.time()
file_path = 'output.txt'
png_to_label(pngimg_array,file_path)
t1 = time.time()
print(t1-t0)


0.15483498573303223
